In [4]:
import win32gui, win32ui, win32con, win32api
import numpy as np
import time
import mss
d,r,c=32,16,30#格子边长,行数,列数
hwnd=win32gui.FindWindow(None,"扫雷")
if hwnd==0:#未打开程序
    win32api.ShellExecute(0,'open',r'C:\Users\KW\me\扫雷.exe','','',1)
    time.sleep(0.2)
    hwnd=win32gui.FindWindow(None,"扫雷")
win32gui.SendMessage(hwnd,win32con.WM_SYSCOMMAND,win32con.SC_RESTORE,0)#还原最小化
mss.mss()#有去除屏幕缩放影响的效果
left,top,right,bottom=win32gui.GetWindowRect(hwnd)
left,top,right,bottom=left+30,top+202,right-22,bottom-22
win32api.keybd_event(13,0,0,0)#模拟键盘输入防止置顶出错
win32gui.SetForegroundWindow(hwnd)#窗口置顶
time.sleep(0.2)
#rgb blank,flag,0,1,2,3,4,5,6,7,8,fail,fail
#a=[589176,516660,566016,514656,449536,486408,465664,440576,473856,464640,507648,206736,391632]
#bgr blank,flag,0,1,2,3,4,5,6,7,8,fail,fail
a=[850296,777780,827136,775776,710656,747528,726784,701696,734976,725760,768768,652752,467856]
m=np.zeros((r+2,c+2))#地图
m[1:r+1,1:c+1]=-2
def new():
    m[1:r+1,1:c+1]=-2
    win32api.keybd_event(113,0,0,0)     
    win32api.keybd_event(113,0,win32con.KEYEVENTF_KEYUP,0)
    time.sleep(0.05)#重开等待时间
def getmap():
    img=np.array(mss.mss().grab((left,top,right,bottom)))
    for i,j in np.argwhere(m<=-2):#上一轮空白格或未知非空白格
        value=np.sum(img[i*d-d:i*d,j*d-d:j*d])
        m[i,j]=np.argwhere(a==value)[0,0]-2
        if m[i,j]>8:
            return -1#失败
    if np.sum(m==-2)==0:#本轮没有空白格
        return 1#成功
    for i,j in np.argwhere(m>0):#去掉无用的数字格子
        if np.sum(m[i-1:i+2,j-1:j+2]==-2)==0:
            m[i,j]=0
    for i,j in np.argwhere(m==-1):#去掉无用的旗子
        if np.sum(m[i-1:i+2,j-1:j+2]==0)==8:
            m[i,j]=0
    return 0
def mouse(i,j,lr):
    x=left+d*j-d//2
    y=top+d*i-d//2
    win32api.SetCursorPos((x,y))
    #time.sleep(0.2)
    if lr=='r':
        m[i,j]=-1#旗子
        win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTDOWN,0,0,0,0)
        win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTUP,0,0,0,0)
    elif lr=='l':
        m[i,j]=-3#暂时未知的非空白格
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0,0,0)
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0,0,0)
def fun0():
    while(1):
        i=np.random.randint(1,r+1)
        j=np.random.randint(1,c+1)
        if m[i,j]==-2:
            mouse(i,j,'l')
            break
def fun1():
    num=[(i,j) for i,j in np.argwhere(m>=1) if np.sum(m[i-1:i+2,j-1:j+2]==-2)>0]#周围有空白格的数字格
    for i,j in num:
        blank=(np.argwhere(m[i-1:i+2,j-1:j+2]==-2)+[i-1,j-1]).tolist()#数字格关联的空白格
        flag=np.sum(m[i-1:i+2,j-1:j+2]==-1)#数字格关联的旗子数
        if m[i,j]==len(blank)+flag:
            for i,j in blank:
                mouse(i,j,'r')
            fun1()
            return 1
        elif m[i,j]==flag:
            for i,j in blank:
                mouse(i,j,'l')
            fun1()
            return 1
    return 0
def fun2():
    num=[(i,j) for i,j in np.argwhere(m>=1) if np.sum(m[i-1:i+2,j-1:j+2]==-2)>0]#周围有空白格的数字格
    for i1,j1 in num:
        for i2,j2 in num:
            if abs(i1-i2)<=2 or abs(j1-j2)<=2:
                b1=(np.argwhere(m[i1-1:i1+2,j1-1:j1+2]==-2)+[i1-1,j1-1]).tolist()#数字格关联的空白格
                b2=(np.argwhere(m[i2-1:i2+2,j2-1:j2+2]==-2)+[i2-1,j2-1]).tolist()
                f1=np.sum(m[i1-1:i1+2,j1-1:j1+2]==-1)#数字格关联的旗子数
                f2=np.sum(m[i2-1:i2+2,j2-1:j2+2]==-1)
                b3=[x for x in b1 if x in b2]
                if len(b3)>=2 and 2*len(b3)!=len(b1)+len(b2):
                    b_1=[x for x in b1 if x not in b3]
                    b_2=[x for x in b2 if x not in b3]
                    if m[i1,j1]-f1-len(b_1)==m[i2,j2]-f2:
                        for i,j in b_2:
                            mouse(i,j,'l')
                        for i,j in b_1:
                            mouse(i,j,'r')
                        fun2()
                        return 1
    return 0
def fun3():
    return 1
def main():
    p,win,fail=10,0,0            
    while(1):
        start = time.time()
        sr=getmap()
        end = time.time()
#         print(end-start)
        if sr==1:
            win+=1
            new()
        elif sr==-1:
            fail+=1
            new()
        if win+fail==p:
            print(win,fail)
            break
        st=fun1()
        if st==0:
            st=fun2()
            if st==0:
                fun0()
if __name__ == '__main__':
#     main()
    getmap()
    fun()


[[[1, 1], [1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2], [3, 3]]]
[[[2, 2]]]


In [1]:
def fun():
    blank=[[i,j] for i,j in np.argwhere(m==-2) if np.sum(m[i-1:i+2,j-1:j+2]>0)>0]#周围有数字格的空白格
    num=[(np.argwhere(m[i-1:i+2,j-1:j+2]>0)+[i-1,j-1]).tolist() for i,j in blank]#空白格分别对应的数字格
    blank1=[]#分段的空白格
    blank2=[]#分段归类的空白格
    num1=[]#分段的空白格对应的数字格
    num2=[]#分段归类的空白格对应的数字格
    b=[blank[0]]
    n1=num[0].copy()
    n2=[num[0]].copy()
    del blank[0]
    del num[0]
    while(len(blank)>0):
        st=0
        for k in range(len(blank)):
            if len([x for x in num[k] if x in n1])>0:#关联条件
                b.append(blank[k])
                t=0
                for x in n2:#归类
                    if x==num[k]:
                        t=1
                if t==0:
                    n2.append(num[k])
                for x in num[k]:
                    if x not in n1:
                        n1.append(x)
                del blank[k]
                del num[k]
                st=1
                break
        if st==0:#开始下一段
            blank1.append(b)
            num1.append(n1)
            num2.append(n2)
            b=[blank[0]]
            n1=num[0].copy()
            n2=[num[0]].copy()
            del num[0]
            del blank[0]
    blank1.append(b)
    num1.append(n1)
    num2.append(n2)
    for k in range(len(blank1)):#获取分段归类的空白格
        b2=[]
        for n2 in num2[k]:
            b=[]
            for i,j in blank1[k]:
                if (np.argwhere(m[i-1:i+2,j-1:j+2]>0)+[i-1,j-1]).tolist()==n2:
                    b.append([i,j])
            b2.append(b)
        blank2.append(b2)
    print(blank2[0])
    print(num2[0])